## Install Minikube

Following command should work for Linux, for other OS, see [Minikube docs](https://github.com/kubernetes/minikube/blob/master/README.md#installation)

```
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64 &&\
sudo install minikube-linux-amd64 /usr/local/bin/minikube &&\
rm minikube-linux-amd64
```

### (Optional) Ensure a clean Minikube cluster

```
minikube delete; rm -rf ~/.minikube
```

## Initialize the Odata Minikube environment

Create the host path for persistent storage

```
sudo mkdir -p /var/odata-minikube-storage
```

Switch to the Odata Minikube environment

```
source switch_environment.sh odata-minikube
```

Initialize the cluster

```
charts-external/odata/deploy.sh --install-minikube
```

## Switch to the Odata Minikube environment

In [ ]:
source switch_environment.sh odata-minikube

## Deploy

You can either Deploy a clean environment from scratch or deploy from back (if you have the required credentials)

### Deploy from scratch

In [ ]:
charts-external/odata/deploy.sh --initialize

### Deploy from backup

You will need a Google service account file with permissions to the odata backups bucket

In [ ]:
# DEPLOY_FROM_BACKUP_SERVICE_ACCOUNT=

charts-external/odata/deploy.sh --restore \
    "${DEPLOY_FROM_BACKUP_SERVICE_ACCOUNT}" \
    gs://odata-k8s-backups/production/blue/ckan-db-dump-$(date +%Y-%m-%d).ckan.dump

### Wait for pods to be in Running state

In [ ]:
kubectl get pods

Some pods might take a while on first deploy due to pulling large Docker images and waiting for other components, please be patient.. it will start eventually

You can use the following commands to inspect the main pods (ckan / datastore-db / db etc..)

In [ ]:
# Describe the first pod serving deployment ckan
./kubectl.sh describe ckan

# Get logs of container secrets in the first pod serving deployment ckan
./kubectl.sh logs ckan -c secrets

# Get yaml of first pod serving deployment ckan
./kubectl.sh get ckan -o yaml

If CKAN API responds it's a good indication that environment was setup successfully

In [ ]:
echo $(./kubectl.sh exec ckan -- wget -qO - http://127.0.0.1:5000/api/3)

It should respond with `{"version": 3}`

## Start port forwarding to access the CKAN web-app

Open a new terminal and run:

```
./kubectl.sh port-forward nginx 5000:80
```

You will need to restart it if CKAN pod is restarted

The web app should be available at http://localhost:5000

## Start the pipelines server

In [ ]:
charts-external/odata/deploy.sh --force --set pipelines.enabled=true

#### Wait for pods to be in Running state

In [ ]:
kubectl get pods

#### Using the pipelines server

In Minikube environment pipelines server is run in manual mode by default, so the pipelines dashboard is not available and scheduled pipelines are disabled.

You can run the pipelines manually

#### List the available pipelines

In [ ]:
./kubectl.sh exec pipelines dpp

#### Run a pipeline

In [ ]:
./kubectl.sh exec pipelines -- dpp run --verbose ./ckanext-odata_org_il/download_foi_offices

#### Check the generated datapackage

In [ ]:
./kubectl.sh exec pipelines -- cat /var/lib/ckan/data/pipelines/odata_org_il/new_foi_offices/datapackage.json | jq

## Local Development of odata ckan

#### Install supervisord

Supervisord is used to run and manage background processes for the development environment

```
sudo easy_install supervisor
```

#### Ensure all components are deployed and working

In [20]:
source switch_environment.sh odata-minikube
./helm_upgrade_external_chart.sh odata

minikube$ Switching to odata-minikube  environment
Switched to context "minikube".
Connected to minikube
minikube$ Switched to context "minikube".
Connected to minikube
RELEASE_NAME=odata-minikube-odata-odata-minikube
CHART_DIRECTORY=charts-external/odata
Release "odata-minikube-odata-odata-minikube" has been upgraded. Happy Helming!
LAST DEPLOYED: Thu Oct 11 17:58:40 2018
NAMESPACE: odata-minikube
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME   AGE
ckan   1d
nginx  1d

==> v1/Service
ckan-jobs-db  1d
ckan          1d
datapusher    1d
datastore-db  1d
db            1d
nginx         1d
redis         1d
solr          1d

==> v1beta1/Deployment
ckan          2s
ckan-jobs-db  2s
ckan-jobs     2s
datastore-db  1d
db            1d
nginx         2s
redis         1d
solr          1d

==> v1/Pod(related)

NAME                           READY  STATUS             RESTARTS  AGE
ckan-6989694f77-zphtx          0/1    Init:0/2           0         1s
ckan-jobs-db-54565fbc7c-4dd9v  0/1    Pending 

: 1

In [40]:
kubectl get pods

minikube$ NAME                            READY     STATUS    RESTARTS   AGE
ckan-6989694f77-zphtx           0/1       Running   0          52s
ckan-jobs-8454ddbbb8-b7nch      2/2       Running   0          52s
ckan-jobs-db-54565fbc7c-4dd9v   1/1       Running   0          52s
datastore-db-56bd444b9c-5zbvw   1/1       Running   0          40s
db-7dff9965dd-mlgrn             1/1       Running   0          44s
nginx-69d85f565f-4pqsq          1/1       Running   0          52s
nginx-69d85f565f-cvt52          1/1       Running   0          52s
redis-688f66b5d4-sm4vr          1/1       Running   0          40m
solr-756d5f5687-fbl5k           1/1       Running   0          12s


: 1

#### Start the supervisord daemon

This starts port forwards for all the infrastructure componnets from localhost to the Minikube cluster

In [ ]:
supervisord -c environments/odata-minikube/local-dev-supervisord.conf

#### Ensure all port forwards are running

In [41]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf status

minikube$ ckan-gunicorn                    STOPPED   Not started
ckan-jobs                        STOPPED   Not started
port-forward-ckan-jobs-db        FATAL     Exited too quickly (process log may have details)
port-forward-datastore-db        RUNNING   pid 20018, uptime 0:38:59
port-forward-db                  RUNNING   pid 20020, uptime 0:38:59
port-forward-redis               RUNNING   pid 20021, uptime 0:38:59
port-forward-solr                RUNNING   pid 20019, uptime 0:38:59


: 1

#### Try a reload if there are problems

In [43]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf reload

minikube$ Restarted supervisord
minikube$ minikube$ ckan-gunicorn                    STOPPED   Not started
ckan-jobs                        STOPPED   Not started
port-forward-ckan-jobs-db        STARTING  
port-forward-datastore-db        STARTING  
port-forward-db                  STARTING  
port-forward-redis               STARTING  
port-forward-solr                STARTING  


: 1

In [45]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf status

minikube$ ckan-gunicorn                    STOPPED   Not started
ckan-jobs                        STOPPED   Not started
port-forward-ckan-jobs-db        RUNNING   pid 27356, uptime 0:00:17
port-forward-datastore-db        RUNNING   pid 27357, uptime 0:00:17
port-forward-db                  RUNNING   pid 27359, uptime 0:00:17
port-forward-redis               RUNNING   pid 27360, uptime 0:00:17
port-forward-solr                RUNNING   pid 27358, uptime 0:00:17


: 1

#### Clone the data4dappl project

In [ ]:
! [ -e ../data4dappl ] && git clone https://github.com/OriHoch/data4dappl.git ../data4dappl

#### Prepare the CKAN configuration file

This copies the configuartion from the minikube ckan pod and modifies some values to make it suitable for local development

In [46]:
CKAN_LOCAL_DEVELOPMENT_INI="environments/odata-minikube/ckan-local-development.ini"
CKAN_LOCAL_DEVELOPMENT_WHO_INI="environments/odata-minikube/who.ini"

# delete the existing file to force recreate of the configuration file
rm -f "${CKAN_LOCAL_DEVELOPMENT_INI}"

if ! [ -e "${CKAN_LOCAL_DEVELOPMENT_INI}" ]; then
    echo Modfying configuration file from pod, saving to "${CKAN_LOCAL_DEVELOPMENT_INI}"
    ./kubectl.sh exec ckan cat /etc/ckan/production.ini \
        | sed -e 's/redis:6379/localhost:6379/g' \
        | sed -e 's/solr:8983/localhost:8983/g' \
        | sed -e 's/db\/ckan/localhost\/ckan/g' \
        | sed -e 's/ckan-jobs-db\/postgres/localhost:5433\/postgres/g' \
        | sed -e 's/datastore-db\/datastore/localhost:5434\/datastore/g' \
            > "${CKAN_LOCAL_DEVELOPMENT_INI}" &&\
    ./kubectl.sh exec ckan cat /etc/ckan/who.ini > "${CKAN_LOCAL_DEVELOPMENT_WHO_INI}"
    if [ "$?" == "0" ]; then
        echo Great Success
    else
        echo Failed to process configuration file
    fi
else
    echo Configuration file already exists, delete to recreate: "${CKAN_LOCAL_DEVELOPMENT_INI}"
fi

minikube$ minikube$ minikube$ minikube$ minikube$ minikube$ minikube$ Modfying configuration file from pod, saving to environments/odata-minikube/ckan-local-development.ini
Great Success


: 1

#### Install the project dependencies

In [ ]:
( cd ../data4dappl; bin/install.sh )

#### Start Gunicorn which serves the CKAN web-app

In [47]:
( cd ../data4dappl
  pipenv run gunicorn \
      --paste ../hasadna-k8s/${CKAN_LOCAL_DEVELOPMENT_INI:-environments/odata-minikube/ckan-local-development.ini} )

minikube$ [2018-10-11 18:01:28 +0000] [27511] [INFO] Starting gunicorn 19.9.0
[2018-10-11 18:01:28 +0000] [27511] [INFO] Listening at: http://0.0.0.0:5000 (27511)
[2018-10-11 18:01:28 +0000] [27511] [INFO] Using worker: sync
[2018-10-11 18:01:28 +0000] [27522] [INFO] Booting worker with pid: 27522
2018-10-11 18:01:29,998 INFO  [ckan.config.environment] Loading static files from public
2018-10-11 18:01:32,966 INFO  [ckan.config.environment] Loading templates from /home/ori/virtualenvs/data4dappl-68r7vFlY/src/ckan/ckan/templates
2018-10-11 18:01:34,573 INFO  [ckan.config.environment] Loading templates from /home/ori/virtualenvs/data4dappl-68r7vFlY/src/ckan/ckan/templates
2018-10-11 18:01:43,789 INFO  [ckan.lib.base]  /group render time 2.739 seconds
2018-10-11 18:01:43,891 INFO  [ckan.lib.base]  /uploads/group/2018-06-04-164928.041018Capture.PNG render time 0.053 seconds
2018-10-11 18:01:44,174 INFO  [ckan.lib.base]  /uploads/group/2018-06-04-164928.041018Capture.PNG render time 0.257 se

Site should be available at http://localhost:5000

#### Start Gunicorn in the background

In [ ]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf start ckan-gunicorn

#### Get Gunicorn logs

In [ ]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf tail ckan-gunicorn

#### Restart Services and port-forwards

Once everything is setup you should run this to ensure services are connected and working properly

Occasionally you might need to run this again if you encounter wierd problems

In [48]:
echo Remove deployments which should run locally
kubectl delete deployment --namespace odata-minikube --context minikube ckan nginx ckan-jobs pipelines

echo Force restart of infrastructure components
for DEPLOYMENT in db datastore-db ckan-jobs-db redis solr; do
    ./force_update.sh $DEPLOYMENT &
done
wait

echo reload gunicorn
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf reload
while [ "5" != "`supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf status | tee -a /dev/stderr | grep RUNNING | wc -l`" ]
do sleep 2; done

echo Start all services
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf start all
while [ "7" != "`supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf status | tee -a /dev/stderr | grep RUNNING | wc -l`" ]
do sleep 2; done

minikube$ Remove deployments which should run locally
minikube$ deployment "ckan" deleted
deployment "nginx" deleted
deployment "ckan-jobs" deleted
Error from server (NotFound): deployments.extensions "pipelines" not found
minikube$ minikube$ Force restart of infrastructure components
minikube$ [1] 27673
[2] 27674
[3] 27675
[4] 27676
[5] 27677
minikube$ error: open /home/ori/.kube/config.lock: file exists
Switched to context "minikube".
Switched to context "minikube".
error: open /home/ori/.kube/config.lock: file exists
Switched to context "minikube".
Switched to context "minikube".
Connected to minikube
Connected to minikube
Connected to minikube
Connected to minikube
Connected to minikube
deployment "solr" patched
deployment "redis" patched
deployment "ckan-jobs-db" patched
deployment "db" patched
deployment "datastore-db" patched
Waiting for rollout to finish: 0 out of 1 new replicas have been updated...
Waiting for rollout to finish: 0 out of 1 new replicas have been updated...
Wai

: 1

Site should be available at http://localhost:5000

## Local Development of Odata CKAN Pipelines

You should run the steps for local development of CKAN first

Create a pipenv for the pipelines

In [ ]:
TEMPFILE=`mktemp`
curl https://raw.githubusercontent.com/OriHoch/ckanext-datapackage_pipelines/master/datapackage_pipelines_ckanext/requirements.txt > $TEMPFILE
( cd ../data4dappl/ckan/ckanext-odata_org_il/ckanext/odata_org_il/pipelines
  pipenv install --python 3 &&\
  pipenv install -r $TEMPFILE &&\
  pipenv run pip install \
      -e 'git+https://github.com/OriHoch/ckanext-datapackage_pipelines#subdirectory=datapackage_pipelines_ckanext&egg=datapackage_pipelines_ckanext'
)
rm $TEMPFILE

#### Set the required env vars to run the pipelines

In [ ]:
export CKAN_API_KEY=`kubectl get secrets --context=minikube --namespace=odata-minikube pipelines-sysadmin -o json | jq -r .data.apikey | base64 -d`
export CKAN_URL='http://localhost:5000'

#### List the available pipelines

In [ ]:
( cd ../data4dappl/ckan/ckanext-odata_org_il/ckanext/odata_org_il/pipelines
  pipenv run dpp )

#### Run a pipeline

In [ ]:
# Force recreate of the foi offices resource
rm -rf /var/lib/ckan/pipelines/odata_org_il/foi_offices

( cd ../data4dappl/ckan/ckanext-odata_org_il/ckanext/odata_org_il/pipelines
  pipenv run dpp run --verbose ./download_foi_offices,./update_foi_offices_resource )